# 40982 - Visión por Computador - Práctica 7 - Entrenando un detector
Isac Añor Santana

## Detector de palmeras
En esta práctica se realiza un entrenamiento personalizado de YOLOv7 para que detecte palmeras.

## Procedimiento

### Obtención de un conjunto de datos o Dataset
La obtención del Dataset la he hecho personalmente sacando fotos de palmeras que se encuentran en el Campus Universitario.

### Anotación del conjunto de datos
La anotación del [conjunto de datos](https://app.roboflow.com/vcp7/palm-tree-detection-0we3t) la he llevado a cabo usando la herramienta [Roboflow](https://app.roboflow.com/login). Antes de subir las imágenes a directamente a Roboflow, las redimensioné. Originalmente, tenían una dimensión de 3024 píxeles de ancho por 4032 de alto, lo que me pareció demasiado. En consecuencia fueron redimensionadas a 756x1008, igualmente, Roboflow posteriormente lleva a cabo un redimensionamiento a 640x640. El conjunto de datos original se encuentra en el directorio Datasets/Raw-Dataset, el redimensionnado en Datasets/Resized-Dataset y el anotado por Roboflow en Datasets/RoboflowPalmTreeAnotatedDataset. El dataset descargado de Roboflow cuenta con 452 imágenes para el conjunto de entrenamiento, 54 para el de validación y 27 para el de test.

### Entrenamiento
Tras crear el entorno virtual correspondiente y descargar yolov7, se mueven los directorios de entrenamiento, test y validación y se ejecuta el siguiente comando.

`python train.py --device 0 --weights yolov7.pt --cfg cfg/training/yolov7.yaml --data data/data.yaml --hyp data/hyp.scratch.custom.yaml --workers 1 --batch-size 4 --epochs 200 --name yolov7-palmeras`

En este caso, se ha puesto a entrenar durante unas dos horas. Al finalizar produce un fichero, best.pt, que contiene los pesos entrenados y se usará como parámetro posteriormente para realizar detecciones

### Prueba del entrenamiento personalizado
Para probar el entrenamiento se toman unas imágenes y videos grabados y se pasan por el detector. Las imágenes estarán bajo el directorio images/ y los vídeos en videos/. La detección se lleva a cabo en el resto del cuaderno. Los resultados se pueden ver bajo el directorio Results.

#### Paquetes necesarios

In [1]:
import cv2
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil

In [ ]:
# Verificación que cuda está disponible
import torch
print(torch.cuda.is_available())


#### Detección en imágenes

In [ ]:
!python yolov7/detect.py --weights best.pt --conf 0.50 --source ./images/ --save-txt --device 0


#### Detección en vídeos

In [ ]:
!python yolov7/detect.py --weights best.pt --conf 0.5 --source videos/ --device 0

Por algún motivo desconocido, le da la vuelta a los vídeos así que toca volver a darles la vuelta. He usado esta entrada de [stackoverflow](https://stackoverflow.com/questions/67507275/rotate-video-180-with-opencv-python).

In [26]:
def flip_video(input_file_path, output_file_path):
    #that's my original video - the one that I want to rotate 180 degrees 
    cap = cv2.VideoCapture(input_file_path)
        
    frame_number = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Get width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # here I'm trying to write the new rotated video
    # Open the output video file before the loop, cv2.VideoWriter_fourcc(*"mp4v") = 0x7634706d
    newvideoR = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc(*"mp4v"), 24, (frame_width, frame_height))
        
    # Original Frames
    #frames = []
    for i in range(frame_number):
        ret, frame = cap.read()
        #frames.append(frame)  # No need to append the original frames

        #here's where I try to rotate the video 
        new = cv2.rotate(frame, cv2.ROTATE_180)
        
        #cv2.imshow('output', new)
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break

        newvideoR.write(new)

    newvideoR.release()
    cap.release()

In [27]:
for input_path in  glob.glob("runs\detect\exp4\*"):
    output_path = "Results\\Videos\\" + os.path.basename(input_path)
    flip_video(input_path, output_path)
    